In [ ]:
import os
import json
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [ ]:
# read all csv in the folder
root_path = ""
original_df = pd.DataFrame()
rephrased_df = pd.DataFrame()

for root, dirs, files in os.walk(root_path):
    # if root.endswith("merged"):
    #     continue
    for file in files:
        if file.endswith(".csv"):
            file_name = file.split(".")[0]
            split_name, mark = file_name.split("_")
            # if split_name == "test":
            #     continue
            if mark.startswith("original"):
                df = pd.read_csv(root + file)
                df["split_name"] = split_name
                print(f"Read {file_name} with shape {df.shape}")
                original_df = pd.concat([original_df, df])
            elif mark.startswith("rephrased"):
                df = pd.read_csv(root + file)
                df["split_name"] = split_name
                print(f"Read {file_name} with shape {df.shape}")
                rephrased_df = pd.concat([rephrased_df, df])
            else:
                print(f"Error: file name: {file_name} is not correct!")
                

In [ ]:
rephrased_df = rephrased_df[rephrased_df["re_idx"] == 0]

feat_cols = list(
    set(original_df.columns) - set(["p_idx", "question_NL", "question_example", "difficulty", "split_name"])
)

# merge two dataframes based on the p_idx
merged_df = pd.merge(original_df, rephrased_df, on=["p_idx", "split_name"], suffixes=("_original", "_rephrased"), ).dropna()

# convert Instruction to one-hot encoding
merged_df = pd.get_dummies(merged_df, columns=["Instruction", "Role", "Scenario"], prefix=["Inst", "Role", "Scen"])
# drop colum "re_idx"
merged_df = merged_df.drop(columns=["Inst_None", "Role_None", "Scen_None"])

instruction_cols = ["long","short","formal","fluent","technical","logical"]
role_cols = ["student","programmer","competitor"]
scenario_cols = ["clearer","improve","specify"]